# "가위바위보 강화학습"
> "두 에이전트가 겨루는 가위바위보 환경에서 강화학습을 통하여 승리 정책을 학습한다."

- toc: true
- badges: true
- author: 단호진
- categories: [rl]

## 가위바위보 환경

In [1]:
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.agents.pg import PGTorchPolicy, PGTrainer
from ray.rllib.examples.env.rock_paper_scissors import RockPaperScissors

torch, _ = try_import_torch()
torch.__version__

Instructions for updating:
non-resource variables are not supported in the long term


'1.7.1+cu110'

* player1과 player2가 취할 수 있는 action은 0, 1, 2이고, 관측치는 상대방의 action에서 나오는 0, 1, 2이다.
* env.reset()은 첫 관측치 {'player1': 0, 'player2': 0}를 내어 준다.
* env.step() 함수는 obs, rewards, done, info를 돌려준다.

In [2]:
env_config = dict()
env = RockPaperScissors(env_config)

In [3]:
obs = env.reset()
obs

{'player1': 0, 'player2': 0}

In [4]:
env.action_space, env.observation_space

(Discrete(3), Discrete(3))

In [5]:
env.step(dict(player1=1, player2=2))

({'player1': 2, 'player2': 1},
 {'player1': -1, 'player2': 1},
 {'__all__': False},
 {})

In [6]:
env.step(dict(player1=2, player2=1))

({'player1': 1, 'player2': 2},
 {'player1': 1, 'player2': -1},
 {'__all__': False},
 {})

In [7]:
env.step(dict(player1=0, player2=2))

({'player1': 2, 'player2': 0},
 {'player1': 1, 'player2': -1},
 {'__all__': False},
 {})

## 정책

정책(Policy)은 에이전트가 관측된 환경, 이전 보상 이력을 참고하여 어떤 행동을 취하면 좋은지 결정한다. 에이전트가 둘인 가위바위보 게임에서 학습 가능한 learned 정책과 RandomMove, BeatLastHeuristic, AlwaysSameHeuristic 정책에서 하나를 뽑아 강확학습을 수행하였다. 가위바위보의 최고 전략은 RandomMove로 learned 정책은 이에 도달하는 결과를 보일 것이고, 그 밖에 BeatLastHeuristic, AlwaysSameHeuristic에 데해서는 쉽게 이길 수 있을 것이다.

새로운 정책을 개발한다면 Policy 클래스를 상속하고 compute_actions에 필요한 로직을 구현하면 된다. RandomMove 정책은 obs_batch에 대해서만 고려하여 배치 크기만큼의 임의 행동을 돌려준다. kwargs에 필수 생성 인자 내용을 정리하였다. 환경의 action_space와 observation_space가 그것이다.

In [8]:
import random
from ray.rllib.policy.policy import Policy
from ray.rllib.policy.view_requirement import ViewRequirement


class RandomMove(Policy):
    """Pick a random move"""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def random_action(self):
        return random.choice([
                RockPaperScissors.ROCK, RockPaperScissors.PAPER,
                RockPaperScissors.SCISSORS,
        ])
    
    def compute_actions(self,
                        obs_batch,
                        state_batches=None,
                        prev_action_batch=None,
                        prev_reward_batch=None,
                        info_batch=None,
                        episodes=None,
                        **kwargs):
        """Returns:
            Tuple:
                actions: [BATCH_SIZE, ACTION_SHAPE]
        """
        return [self.random_action() for _ in obs_batch], [], {}

In [9]:
kwargs = {
    'observation_space': env.observation_space,
    'action_space': env.action_space,
    'config': {}
}
rm = RandomMove(**kwargs)

In [10]:
rm.compute_actions(list(range(3)))

([0, 2, 2], [], {})

## 트레이너 설정 및 학습

앞서 정의한 RandomMove외에 ray에서 제공하는 BeatLastHeuristic, AlwaysSameHeuristic 정책을 추가하였다.

In [11]:
import ray
from gym.spaces import Discrete
from ray.rllib.agents.registry import get_trainer_class
from ray.rllib.examples.policy.rock_paper_scissors_dummies import (
    BeatLastHeuristic, AlwaysSameHeuristic
)


config = {
    'env': RockPaperScissors,
    'gamma': 0.9,
    'num_gpus': 0,
    'num_workers': 0,
    'num_envs_per_worker': 4,
    'train_batch_size': 200,  # for the policy model
    'multiagent': {
        'policies': {
            'random_move': (RandomMove, Discrete(3), Discrete(3), {}),
            'beat_last': (BeatLastHeuristic, Discrete(3), Discrete(3), {}),
            'always_same': (AlwaysSameHeuristic, Discrete(3), Discrete(3), {}),
            'learned': (None, Discrete(3), Discrete(3), {
                'model': {},  # use default
                'framework': 'torch'
            })
        },
        'policy_mapping_fn': lambda agent_id: (
            'learned' if agent_id == 'player1' else 'beat_last'),
        'policies_to_train': ['learned'],
    },
    'framework': 'torch',
}

# ray.shutdown()
ray.init()
trainer = get_trainer_class('PG')(config=config)

2021-02-15 07:33:57,811	INFO services.py:1193 -- View the Ray dashboard at http://127.0.0.1:8265
2021-02-15 07:33:59,214	INFO trainer.py:650 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


In [12]:
for _ in range(1):
    results = trainer.train()

In [13]:
list(k for k in results)

['episode_reward_max',
 'episode_reward_min',
 'episode_reward_mean',
 'episode_len_mean',
 'episodes_this_iter',
 'policy_reward_min',
 'policy_reward_max',
 'policy_reward_mean',
 'custom_metrics',
 'hist_stats',
 'sampler_perf',
 'off_policy_estimator',
 'num_healthy_workers',
 'timesteps_total',
 'timers',
 'info',
 'done',
 'episodes_total',
 'training_iteration',
 'experiment_id',
 'date',
 'timestamp',
 'time_this_iter_s',
 'time_total_s',
 'pid',
 'hostname',
 'node_ip',
 'config',
 'time_since_restore',
 'timesteps_since_restore',
 'iterations_since_restore',
 'perf']

In [14]:
for k in range(50):
    results = trainer.train()
    print(k,
          results['episode_reward_mean'],
          results['policy_reward_mean'],
          '\t', results['timesteps_total'],
          '\t', results['episodes_total'])

0 0.0 {'learned': 0.14, 'beat_last': -0.14} 	 1600 	 160
1 0.0 {'learned': 0.21, 'beat_last': -0.21} 	 2400 	 240
2 0.0 {'learned': 0.55, 'beat_last': -0.55} 	 3200 	 320
3 0.0 {'learned': 0.55, 'beat_last': -0.55} 	 4000 	 400
4 0.0 {'learned': -0.28, 'beat_last': 0.28} 	 4800 	 480
5 0.0 {'learned': 0.25, 'beat_last': -0.25} 	 5600 	 560
6 0.0 {'learned': 0.42, 'beat_last': -0.42} 	 6400 	 640
7 0.0 {'learned': 0.71, 'beat_last': -0.71} 	 7200 	 720
8 0.0 {'learned': 0.5, 'beat_last': -0.5} 	 8000 	 800
9 0.0 {'learned': 0.52, 'beat_last': -0.52} 	 8800 	 880
10 0.0 {'learned': 0.26, 'beat_last': -0.26} 	 9600 	 960
11 0.0 {'learned': 0.08, 'beat_last': -0.08} 	 10400 	 1040
12 0.0 {'learned': 1.15, 'beat_last': -1.15} 	 11200 	 1120
13 0.0 {'learned': 0.45, 'beat_last': -0.45} 	 12000 	 1200
14 0.0 {'learned': 0.37, 'beat_last': -0.37} 	 12800 	 1280
15 0.0 {'learned': 0.79, 'beat_last': -0.79} 	 13600 	 1360
16 0.0 {'learned': 0.96, 'beat_last': -0.96} 	 14400 	 1440
17 0.0 {'learn

학습이 진행되면서 learned 정책이 BeatLastHeuristic 정책을 쉽게 이기는 것을 알 수 있다.

## 맺으며

ray의 rllib은 다양한 강화학습 방식을 지원할뿐만 아니라 병렬 학습에 대한 처리가 매우 우수하다. 게다가 활발하게 코드가 관리되고 있다. 다만, 사용자 환경이나 모델을 정의할 때 생각대로 되지 않았던 기억이 남아있는데 향후의 블로그에서 관련 내용을 추가해 보겠다.